In [ ]:
import numpy as np
import os
import tensorflow as tf

from joblib import load

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)

import sys
sys.path.append("../")
%load_ext autoreload
%autoreload 2

In [ ]:
########################################
########### Prepare Data ###############
########################################
from modules.data_processor import generate_data

root_path = '../../data/'
datasets= ['character_trajectories/dataset_steps-20_timesteps-206.pickle', 'anomaly_new/anomaly_dataset.pickle', 'FordA/dataset_classes-2_timesteps-500.pickle', 'ElectricDevices/dataset_classes-7_timesteps-96.pickle', 'daily_and_sport_activites/dataset_classes-19_timesteps-60.pickle']
path = os.path.join(root_path, datasets[1])
trainX, trainY, valX, valY, testX, testY, classes, seqlen, channel = generate_data(path, create_val=True, verbose=1)

trainLen, valLen, testLen = trainX.shape[0], valX.shape[0], testX.shape[0]

set_name = path.split(os.sep)[-2]
model_path = os.path.join('../../models', set_name)
img_path = os.path.join('../../images', set_name)
if not os.path.exists(model_path):
    os.makedirs(model_path)
if not os.path.exists(img_path):
    os.makedirs(img_path)

In [ ]:
# not modularized

def validate_and_adjust_settings(zero, attach, notemp):
    # 0 0 0 invalid
    # 1 0 0 valid
    # 0 1 0 valid
    # 1 1 0 valid
    # 0 0 1 invalid
    # 1 0 1 valid
    # 0 1 1 invalid
    # 1 1 1 valid
    if zero == 0 and attach == 0:
        return 1, attach, notemp
    if attach == 0 and notemp == 0:
        return 1, attach, notemp
    if notemp == 1:
        return 1, attach, notemp
    return zero, attach, notemp

def define_setup(config, zero, attach, notemp):
    s = 'strides_'
    l = 'length_'
    for c in config:
        s += str(c[0]) + '-'
        l += str(c[1]) + '-'
    s = s[:-1] + '_' + l[:-1] +'_zero-'
    s += '1' if zero else '0'
    s += '_attach-'
    s += '1' if attach else '0'
    s += '_notemp-' 
    s += '1' if notemp else '0'
    return s

In [ ]:
########################################
############# Patch Data ###############
########################################
from modules.data_generator import DataGenerator
from modules.patch_generator import get_generator_id_list

# [Stride, Length]
config = [[5,10]]
zero, attach, notemp = True, True, False
zero, attach, notemp = validate_and_adjust_settings(zero, attach, notemp)

params = {'dim': [seqlen, channel], 'batch_size': 1024, 'config': config,
          'zero': zero, 'attach': attach, 'notemp': notemp, 'shuffle': False}

clf_type= 'svm'

setup = define_setup(config, zero, attach, notemp)
setup_path = os.path.join(model_path, setup)
if not os.path.exists(setup_path):
    os.makedirs(setup_path)
image_path = os.path.join(img_path, setup, clf_type)
if not os.path.exists(image_path):
    os.makedirs(image_path)

# Generators
trainIds = get_generator_id_list(trainLen, seqlen, config)
train_generator = DataGenerator(trainIds, trainX, trainY, **params)
valIds = get_generator_id_list(valLen, seqlen, config)
val_generator = DataGenerator(valIds, valX, valY, **params)
testIds = get_generator_id_list(testLen, seqlen, config)
test_generator = DataGenerator(testIds, testX, testY, **params)

In [ ]:
########################################
############ Train Level 1 #############
########################################
from modules.model import create_model
from modules.model_trainer import train_descriptive

input_shape = trainX.shape[1:]
if attach:
    input_shape = list(input_shape)
    input_shape[-1] +=1
    input_shape = tuple(input_shape)
patch_model_path = os.path.join(setup_path, 'patch_classifier.h5')

if os.path.exists(patch_model_path):
    patch_model = tf.keras.models.load_model(patch_model_path)
else:
    patch_model = create_model(input_shape, classes)
    patch_model = train_descriptive(patch_model_path, patch_model, trainIds, valIds, trainX, trainY, valX, valY, params, thresh=0.0, verbose=1, workers=1)

softmax_trainXp = patch_model.predict(train_generator)[:len(trainIds)]
softmax_valXp = patch_model.predict(val_generator)[:len(valIds)]
softmax_testXp = patch_model.predict(test_generator)[:len(testIds)]


In [ ]:
########################################
############ Train Level 2 #############
########################################
from modules.patch_generator import get_sample_id_list, create_histo_dataset, get_data_patch_stats
from modules.model import create_clf
from modules.model_trainer import train_clf

train_pps = get_data_patch_stats(trainLen, seqlen, config)[1]
train_sidx = get_sample_id_list(trainLen, train_pps)
val_pps = get_data_patch_stats(valLen, seqlen, config)[1]
val_sidx = get_sample_id_list(valLen, val_pps)
test_pps = get_data_patch_stats(testLen, seqlen, config)[1]
test_sidx = get_sample_id_list(testLen, test_pps)

histo_trainX = create_histo_dataset(softmax_trainXp, train_sidx)
histo_valX = create_histo_dataset(softmax_valXp, val_sidx)
histo_testX = create_histo_dataset(softmax_testXp, test_sidx)

clf_model_path = os.path.join(setup_path, clf_type + '_classifier.pickle')
if os.path.exists(clf_model_path):
    clf = load(clf_model_path)
else:
    clf = create_clf(clf_type)
    clf = train_clf(clf_model_path, clf, histo_trainX, trainY, histo_valX, valY)

clf_train_pred = clf.predict(histo_trainX)
clf_val_pred = clf.predict(histo_valX)
clf_test_pred = clf.predict(histo_testX)

In [ ]:
########################################
########### Plot Statistics ############
########################################
from modules.patch_generator import get_all_patch_params, get_data_patch_stats, get_all_patch, get_patch_params_list
from modules.plot_processor import plot_heatmap, plot_class_means, plot_series_and_dist, plot_patch_and_dist, plot_class_overlay

# patch + dist
idx = 0
show_patches = [0]

npc, pps = get_data_patch_stats(trainLen, seqlen, config)
ids = get_all_patch_params(idx, npc, pps)
samples = get_all_patch(ids, trainX, trainLen, seqlen, config, zero, attach, notemp)

patch_dists = patch_model.predict(samples)
param_list = get_patch_params_list(ids, trainLen, seqlen, config)

for i in show_patches:
    plot_patch_and_dist(idx, trainX[idx], trainY[idx], np.argmax(patch_dists[i]), patch_dists[i], param_list[i], patch=i)#, save=image_path)

# complete sample
idx = 0

npc, pps = get_data_patch_stats(trainLen, seqlen, config)
ids = get_all_patch_params(idx, npc, pps)
samples = get_all_patch(ids, trainX, trainLen, seqlen, config, zero, attach, notemp)

patch_preds = patch_model.predict(samples)
plot_series_and_dist(idx, trainX[idx], trainY[idx], clf_train_pred[idx], patch_preds)#, save=image_path)

# class overlay
idx = 0
only_classes = [0]
ids = get_all_patch_params(idx, npc, pps)
samples = get_all_patch(ids, trainX, trainLen, seqlen, config, zero, attach, notemp)

patch_dists = patch_model.predict(samples)
param_list = get_patch_params_list(ids, trainLen, seqlen, config)
plot_class_overlay(idx, trainX[idx], trainY[idx], clf_train_pred[idx], patch_dists, param_list, only_classes=only_classes)#, save=image_path)

# class means
#plot_heatmap(train_class_means)
#plot_class_means(np.expand_dims(train_class_means[0], axis=0))

In [ ]:
# not modularized

def process_pipline(data, l1_model, l2_model, config, params, labels=None):
    # data shapes
    if len(data.shape) < 3:
        data = np.expand_dims(data, axis=0)
        if not labels is None:
            labels= np.array([labels])
    dataLen, seqLen, channels = data.shape
    dataIds = get_generator_id_list(dataLen, seqLen, config)
    # data generator
    if labels is None:
        data_generator = DataGenerator(dataIds, data, -np.ones(data.shape[0]), **params)
    else:
        data_generator = DataGenerator(dataIds, data, labels, **params)
    # level 1
    softmax_dataXp = l1_model.predict(data_generator)[:len(dataIds)]
    # level 2
    data_pps = get_data_patch_stats(dataLen, seqLen, config)[1]
    data_sidx = get_sample_id_list(dataLen, data_pps)
    histo_dataX = create_histo_dataset(softmax_dataXp, data_sidx)
    clf_data_pred = l2_model.predict(histo_dataX)
    # statistics
    if not labels is None:
        get_classification_report(labels, clf_data_pred, complete=True)
    
    return softmax_dataXp, histo_dataX, clf_data_pred

def sample_modifier(src_data, target_data, src_ids, target_ids, src_channel=None, target_channel=None):
    result = np.copy(target_data)
    for i in range(len(src_ids)):
        if src_channel is None:
            result[target_ids[i][0]:target_ids[i][1]] = src_data[src_ids[i][0]:src_ids[i][1]]
        else:
            result[target_ids[i][0]:target_ids[i][1], target_channel[i]] = src_data[src_ids[i][0]:src_ids[i][1], src_channel[i]]
    return result

def compare_explanation(sample, patch_model, clf, config, params, blackbox_model, label):
    soft_pred, histo_data, clf_pred = process_pipline(sample, patch_model, clf, config, params)
    black_pred = blackbox_model.predict(np.expand_dims(sample, axis=0))
    plot_series_and_dist(0, sample, label, clf_pred[0], soft_pred)

In [ ]:
# not modularized 

idx = 1
sample = trainX[idx]
label = trainY[idx]

steps = [1, 1.125, 1.25, 1.375, 1.5, 1.625, 1.75]

series = []
for i in steps:
    m_sample = np.copy(sample)
    m_sample[12,1] -= i
    if m_sample[11,1] - m_sample[12,1] > 2:
        label = 1
    #compare_explanation(m_sample, patch_model, clf, config, params, blackbox_model, label)
    soft_pred, histo_data, clf_pred = process_pipline(m_sample, patch_model, clf, config, params)
    series.append([m_sample, label, soft_pred, clf_pred])

In [ ]:
# not modularized 

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import LineCollection

fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')

window = np.arange(5,21)
zs = np.arange(len(series))
ys = []
for i in range(len(series)):
    s = series[(len(series) -1) - i]
    ts = s[0][window]
    ys.append([[idx, ts[idx][1]] for idx in range(len(ts))])
lines = LineCollection(ys, color=['C' + str(series[(len(series) -1) - i][1]) for i in range(len(series))])
lines.set_alpha(0.7)
ax.add_collection3d(lines, zs=zs, zdir='y')

if 1:
    patch2v = [[0,11], [5,16]]
    patch2vid = [1,2]
    for c in range(len(series)):
        for i in range(len(patch2v)):
            px = np.arange(patch2v[i][0], patch2v[i][1], 10)
            s1 = (i+1) % 2
            s2 = i % 2
            py = np.array([(len(series) - 1) - c-0.4*s1, (len(series) - 1) - c+0.4*s2])
            px, py = np.meshgrid(px, py)
            pz = np.ones(np.ravel(px).shape) * -1.5
            pz = pz.reshape(px.shape)
            max_class = np.argmax(series[c][2][patch2vid[i]])
            col = 'C' + str(max_class)
            edge_col = 'C' + str(series[c][3][0])
            ax.plot_surface(px, py, pz, color=col, edgecolor=edge_col, linewidth=3, alpha=np.max(series[c][2][patch2vid[i]]))


ax.set_xlabel('Patch',  fontsize=16)
ax.set_xticklabels(['', '1', '', '', '', '2', ''])
ax.set_ylabel('Change',  fontsize=16)
ax.set_zlabel('Softmax',  fontsize=16)

fig.suptitle('Increased peak size over time',  fontsize=16)

ax.set_xlabel('Timestep',  fontsize=16)
ax.set_xlim3d(0,len(window))
ax.set_xticks(np.arange(0, len(window)+1, 5))
ax.set_xticklabels(window[::5])
ax.set_ylabel('Change',  fontsize=16)
ax.set_yticks(np.arange(len(series)))
ax.set_yticklabels(np.arange(len(series))[::-1])
ax.set_ylim3d(-.4, 6.4)
ax.set_zlabel('Value',  fontsize=16)
ax.set_zlim3d(-1.5, 2)

fig.subplots_adjust(top=0.99)
ax.view_init(elev=20, azim=30)#elev=10, 
#plt.savefig(os.path.join(image_path, 'Class_Smoothing'), dpi=300)
plt.show()

In [ ]:
# not modularized 

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

window = np.array([1,2])
for i in range(len(series)):
    s = series[i]
    xs = np.arange(len(s[2]))[window]
    ys = np.max(s[2], axis=1)[window]
    c = np.argmax(s[2][window], axis=1)
    c = np.array(['C' + str(i) for i in c])

    z = np.repeat(i, len(xs))

    ax.bar(xs, ys, zs=z, zdir='y', color=c, edgecolor='black' , alpha=0.5)

ax.set_xlabel('Patch',  fontsize=16)
ax.set_xticklabels(['', '1', '', '', '', '2', ''])
ax.set_ylabel('Change',  fontsize=16)
ax.set_zlabel('Softmax',  fontsize=16)

fig.suptitle('Patch label change over time',  fontsize=16)

fig.subplots_adjust(top=0.99)
ax.view_init(azim=45)#elev=10, 
#plt.savefig(os.path.join(image_path, 'Class_Smoothing_bars'), dpi=300)
plt.show()